In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [5]:
conn = sq.connect('../../../../Data-Pipelines/Outputs/CensusBureau.db')
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52021
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_1]', conn)
ACS52021_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_2]', conn)
ACS52021_2 = pd.DataFrame(sql_query)
ACS52021 = ACS52021_1.merge(ACS52021_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52021.drop(columns = 'Source', inplace = True)
dfs2021 = [ACS52021, PL2020]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)

In [11]:
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
           'Humphreys County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 'Rutherford County, Tennessee', 
           'Sumner County, Tennessee', 'Stewart County, Tennessee', 'Trousdale County, Tennessee', 'Williamson County, Tennessee', 
           'Wilson County, Tennessee', 'GNRC']
data =   twothousandtwentyone.loc[twothousandtwentyone['NAME'].isin(thelist)].reset_index(drop = True)

In [13]:
cols = ['NAME', 'HHIncome%:Less than 10,000', 'HHIncome%:10 to 14,999', 'HHIncome%:15 to 19,999', 'HHIncome%:20 to 24,999', 'HHIncome%:25 to 29,999', 
        'HHIncome:%30 to 34,999', 'HHIncome%:35 to 39,999', 'HHIncome%:40 to 44,999', 'HHIncome%:45 to 49,999', 'HHIncome%:50 to 59,999', 
        'HHIncome%:60 to 74,999', 'HHIncome%:75 to 99,999', 'HHIncome%:100 to 124,999', 'HHIncome%:125 to 149,999', 
        'HHIncome%:150 to 199,999', 'HHIncome%:200K or More', 'HealthCoverage%: None', 'Total Households']
data = data[cols]
data.head(2)

,NAME,"HHIncome%:Less than 10,000","HHIncome%:10 to 14,999","HHIncome%:15 to 19,999","HHIncome%:20 to 24,999","HHIncome%:25 to 29,999","HHIncome:%30 to 34,999","HHIncome%:35 to 39,999","HHIncome%:40 to 44,999","HHIncome%:45 to 49,999","HHIncome%:50 to 59,999","HHIncome%:60 to 74,999","HHIncome%:75 to 99,999","HHIncome%:100 to 124,999","HHIncome%:125 to 149,999","HHIncome%:150 to 199,999",HHIncome%:200K or More,HealthCoverage%: None,Total Households
0,"Cheatham County, Tennessee",3.514252,2.160614,5.160745,3.520760,2.329819,4.685670,3.657425,5.557725,2.935051,7.503579,13.946375,16.959521,9.189119,9.228166,6.143434,3.507744,10.360394,16590.0
1,"Davidson County, Tennessee",5.893643,3.189285,3.410479,3.701095,4.119663,3.998857,4.442267,4.151311,4.115239,8.360132,10.625164,13.120238,9.444666,6.052222,6.815854,8.559888,12.226077,324727.0


In [14]:
thelist = [data['HHIncome%:60 to 74,999'], data['HHIncome%:75 to 99,999'], data['HHIncome%:100 to 124,999'], data['HHIncome%:125 to 149,999'], 
           data['HHIncome%:150 to 199,999'], data['HHIncome%:200K or More']]
data['% Above 60,000'] = sum(thelist)
thelist = [data['HHIncome%:75 to 99,999'], data['HHIncome%:100 to 124,999'], data['HHIncome%:125 to 149,999'], 
           data['HHIncome%:150 to 199,999'], data['HHIncome%:200K or More']]
data['% Above 75,000'] = sum(thelist)
thelist = [data['HHIncome%:35 to 39,999'], data['HHIncome:%30 to 34,999'], data['HHIncome%:25 to 29,999'], data['HHIncome%:20 to 24,999'], 
           data['HHIncome%:15 to 19,999'], data['HHIncome%:10 to 14,999'], data['HHIncome%:Less than 10,000']]
data['% Below 39,999'] = sum(thelist)
thelist = [data['HHIncome%:40 to 44,999'], data['HHIncome%:35 to 39,999'], data['HHIncome:%30 to 34,999'], data['HHIncome%:25 to 29,999'], data['HHIncome%:20 to 24,999'], 
           data['HHIncome%:15 to 19,999'], data['HHIncome%:10 to 14,999'], data['HHIncome%:Less than 10,000']]
data['% Below 44,999'] = sum(thelist)

In [15]:
cols = ['NAME',  '% Above 60,000', '% Above 75,000', '% Below 39,999', '% Below 44,999', 'HealthCoverage%: None', 'Total Households']
data = data[cols]
data.head(2)

,NAME,"% Above 60,000","% Above 75,000","% Below 39,999","% Below 44,999",HealthCoverage%: None,Total Households
0,"Cheatham County, Tennessee",58.974359,45.027984,25.029285,30.587010,10.360394,16590.0
1,"Davidson County, Tennessee",54.618031,43.992867,28.755287,32.906598,12.226077,324727.0


In [16]:
data

,NAME,"% Above 60,000","% Above 75,000","% Below 39,999","% Below 44,999",HealthCoverage%: None,Total Households
0,"Cheatham County, Tennessee",58.974359,45.027984,25.029285,30.587010,10.360394,16590.0
1,"Davidson County, Tennessee",54.618031,43.992867,28.755287,32.906598,12.226077,324727.0
2,"Dickson County, Tennessee",52.270762,39.358185,29.786909,34.409805,9.220929,22364.0
3,"Houston County, Tennessee",35.629371,27.342657,44.055944,48.776224,10.212713,3967.0
4,"Humphreys County, Tennessee",40.206340,30.068780,37.828947,42.464115,8.740622,8866.0
5,"Montgomery County, Tennessee",52.705913,42.019107,29.047250,33.986574,8.213670,84669.0
6,"Robertson County, Tennessee",55.162798,44.776907,27.709527,32.288212,8.941769,28179.0
7,"Rutherford County, Tennessee",59.337565,48.103064,23.386154,27.864318,9.181438,127965.0
8,"Stewart County, Tennessee",44.038499,34.531526,38.185032,45.629542,9.434948,6688.0
9,"Sumner County, Tennessee",60.921281,48.978496,24.622099,28.028029,7.962217,77407.0
